Instalación de dependencias

In [1]:
pip install folium

Dependencias

In [2]:
import folium
import json
import csv
import requests
import pandas as pd
import shapely
from shapely.geometry import shape, GeometryCollection, Point, Polygon

In [3]:
vAPIKEY=''
with open('mapboxkeys.csv', mode='r', ) as csv_file:
        csv_reader = csv.DictReader(csv_file, delimiter=',')
        for row in csv_reader:
            vAPIKEY=row["APIKEY"]

In [4]:
with open('seccensales.JSON') as f:
    seccensales_geojson = json.load(f)

In [5]:
def get_coordinates_from_string(street_name, apikey):
    URL = "https://api.mapbox.com/geocoding/v5/mapbox.places/"+str(street_name)+".json?access_token="+apikey
    r = requests.get(url = URL)
    data = r.json()
    return data['features'][0]['geometry']['coordinates']

In [6]:
def get_seccensal(lat,long, infeatures):
  seccensal = 0
  point1 = Point(long, lat)
  for feature in infeatures["features"]:
      poly = Polygon(feature["geometry"]["coordinates"][0])
      if poly.contains(point1) == True:
          seccensal = feature["properties"]["coddistsec"]
          break
  return seccensal

Aplicación, Cambiar la dirección sobre la que queremos calcular

In [7]:
street_name = "Regne de valencia 85, Valencia, Valencia"

In [8]:
coordinates = get_coordinates_from_string(street_name, vAPIKEY)

In [9]:
laundry_seccensal = get_seccensal(coordinates[1], coordinates[0], seccensales_geojson)

In [10]:
vlc_demodata = pd.read_csv('dataset_ine_valencia.csv', sep=',', decimal='.')

In [11]:
vlc_demodata.rename(columns = {'Code':'coddistsec'}, inplace = True)

In [12]:
seccensales_geojson

{'crs': {'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'},
  'type': 'name'},
 'features': [{'geometry': {'coordinates': [[[-0.33751415838652,
       39.3852999102882],
      [-0.336081147268443, 39.385604848799495],
      [-0.335255802662981, 39.38344184068258],
      [-0.334462251304255, 39.38353172120158],
      [-0.334409366638321, 39.385129541206226],
      [-0.33204495204898, 39.38489503554787],
      [-0.331798225822192, 39.38689378234773],
      [-0.331372392909679, 39.38905955061944],
      [-0.330808631609023, 39.39038884632363],
      [-0.33089641782236, 39.39145386533725],
      [-0.329946576355461, 39.391130228210955],
      [-0.329330910111584, 39.39174665159683],
      [-0.328767508533673, 39.39240030831035],
      [-0.328091948146294, 39.39765920172644],
      [-0.326416997438565, 39.389576072737746],
      [-0.323065550018189, 39.380107400810175],
      [-0.319177429510942, 39.368856070969784],
      [-0.317029759979862, 39.363851767200835],
      [-0.3123924250

In [13]:
for idx in range(len(seccensales_geojson['features'])):
  seccensales_geojson['features'][idx]['properties']['coddistsec'] = int(seccensales_geojson['features'][idx]['properties']['coddistsec'])

In [14]:
seccensales_geojson

{'crs': {'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'},
  'type': 'name'},
 'features': [{'geometry': {'coordinates': [[[-0.33751415838652,
       39.3852999102882],
      [-0.336081147268443, 39.385604848799495],
      [-0.335255802662981, 39.38344184068258],
      [-0.334462251304255, 39.38353172120158],
      [-0.334409366638321, 39.385129541206226],
      [-0.33204495204898, 39.38489503554787],
      [-0.331798225822192, 39.38689378234773],
      [-0.331372392909679, 39.38905955061944],
      [-0.330808631609023, 39.39038884632363],
      [-0.33089641782236, 39.39145386533725],
      [-0.329946576355461, 39.391130228210955],
      [-0.329330910111584, 39.39174665159683],
      [-0.328767508533673, 39.39240030831035],
      [-0.328091948146294, 39.39765920172644],
      [-0.326416997438565, 39.389576072737746],
      [-0.323065550018189, 39.380107400810175],
      [-0.319177429510942, 39.368856070969784],
      [-0.317029759979862, 39.363851767200835],
      [-0.3123924250

In [15]:
vlc_demodata.dtypes

Unnamed: 0                         int64
Asalariados                      float64
Pensionistas                     float64
Desempleados                     float64
Prestacionistas                  float64
Otros_ingresos                   float64
coddistsec                         int64
Index_Gini                       float64
dist_rent_p80_p20                float64
media_edad                       float64
percent_menos_18                 float64
percernt_mas_65                  float64
tamaño_casa_media                float64
percent_casa_unipersonales       float64
habitantes                       float64
Renta neta media por persona     float64
renta_media_hogar                float64
media_unidad_consumo             float64
mediana_unidad_consumo           float64
renta_bruta_persona              float64
renta_bruta_hogar                float64
viviendas_turisticas             float64
plazas                           float64
plaza_vivienda_turistica         float64
percent_vivienda

In [16]:
map = folium.Map(location=[39.46994829189022, -0.37787440832473984], zoom_start=12)

folium.Choropleth(
    geo_data=seccensales_geojson,
    data=vlc_demodata,
    name ='choropleth', 
    columns=["coddistsec", "habitantes"],
    key_on="feature.properties.coddistsec",
    fill_color='YlGnBu', 
    fill_opacity=0.70, 
    line_opacity=1,
    legend_name='Habitantes',
    smooth_factor=0
).add_to(map)

map